In [56]:
import pandas as pd
import glob
import numpy as np

In [57]:
def fix_dates(df):
    df["DATUM"] = df["MESS_DATUM"].astype(str)
    df["DATUM"] = pd.to_datetime(df["DATUM"].apply(lambda x: x[0:4] + "-" + x[4:6] + "-" + x[6:8] + "-" + x[8:10]))
    df.set_index("DATUM", inplace = True)
    return df

In [58]:
def rename_columns(df):
    # Hier die gewünschten Parameternamen ergänzen
    renames = {
        
    }

    df.rename(columns = renames, inplace = True)
    return df

In [59]:
def create_cleans():

    path = "Daten/Wetter_2024/"
    all_files = glob.glob(path + "schnarrenberg_dwd*.csv")

    dfs = []
    for file in all_files:
        df = pd.read_csv(file, delimiter=";", header=0)
        df = fix_dates(df)
        df = rename_columns(df)
        df.to_csv(f"Daten/Wetter_2024/clean_{file.split("\\")[-1]}" )


In [60]:
def remove_duplicate_columns(df):
    df = df.drop(columns=["eor", "STATIONS_ID", "MESS_DATUM"])
    return df

In [61]:
def create_master():
    path = "Daten/Wetter_2024/"
    all_files = glob.glob(path + "clean_schnarrenberg_dwd*.csv")
    date_range = pd.date_range(start='2023-01-01', end='2025-12-31 23:00:00', freq='h')
    df = pd.DataFrame(index=date_range) 
    for file in all_files:
        df_new = pd.read_csv(file, delimiter=",", header=0)
        df_new = remove_duplicate_columns(df_new)
        df_new["DATUM"] = pd.to_datetime(df_new["DATUM"])
        df_new.set_index("DATUM", inplace = True)
        df = pd.merge(df, df_new, how = "left", left_index = True, right_index = True, suffixes=("", "_y"))
    return df

In [62]:
create_cleans()

In [63]:

merged = create_master()

In [64]:
merged = merged.dropna(how = "all")

In [65]:
merged.to_csv("Daten/Wetter_2024/clean_wetter_komplett.csv")